In [119]:
import pyspark
from pyspark.sql import SparkSession, Row, Column
from pyspark.sql.functions import upper, pandas_udf, expr
import pandas as pd
from datetime import datetime, date

In [2]:
spark = SparkSession.builder.getOrCreate()

23/04/28 14:48:00 WARN Utils: Your hostname, drice resolves to a loopback address: 127.0.1.1; using 192.168.37.109 instead (on interface wlp0s20f3)
23/04/28 14:48:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 14:48:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
#List of Rows
df = spark.createDataFrame([
    Row(a=1,b=2.,c='string1',d=date(2001,1,1),e=datetime(2001,1,1,12,0)),
    Row(a=2,b=3.,c='string2',d=date(2001,2,1),e=datetime(2001,2,1,12,0)),
    Row(a=3,b=4.,c='string3',d=date(2001,3,1),e=datetime(2001,3,1,12,0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [28]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2001-01-01|2001-01-01 12:00:00|
|  2|3.0|string2|2001-02-01|2001-02-01 12:00:00|
|  3|4.0|string3|2001-03-01|2001-03-01 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [29]:
#Explicit schema
df = spark.createDataFrame([
    (1,2.,'string1', date(2000,1,1),datetime(2000,1,1,0)),
    (2,3.,'string2', date(2000,1,1),datetime(2000,1,1,0)),
    (3,4.,'string3', date(2000,1,1),datetime(2000,1,1,0))
],
schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [30]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 00:00:00|
|  2|3.0|string2|2000-01-01|2000-01-01 00:00:00|
|  3|4.0|string3|2000-01-01|2000-01-01 00:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [31]:
#From Pandas
pd_df = pd.DataFrame({
    'a':[1,2,3],
    'b':[2.,3.,4.],
    'c':['string1','string2','string3'],
    'd':[date(2000,1,1),date(2000,2,1),date(2000,3,1)],
    'e':[datetime(2000,1,1,12,0),datetime(2000,2,1,12,0),datetime(2000,3,1,12,0)]
})
df = spark.createDataFrame(pd_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [32]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-02-01 12:00:00|
|  3|4.0|string3|2000-03-01|2000-03-01 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [33]:
#Sumary
df.describe().show()

+-------+---+---+-------+
|summary|  a|  b|      c|
+-------+---+---+-------+
|  count|  3|  3|      3|
|   mean|2.0|3.0|   null|
| stddev|1.0|1.0|   null|
|    min|  1|2.0|string1|
|    max|  3|4.0|string3|
+-------+---+---+-------+



In [34]:
#Collect
df.collect()

[Row(a=1, b=2.0, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3.0, c='string2', d=datetime.date(2000, 2, 1), e=datetime.datetime(2000, 2, 1, 12, 0)),
 Row(a=3, b=4.0, c='string3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 3, 1, 12, 0))]

In [35]:
#Take
df.take(1)

[Row(a=1, b=2.0, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0))]

In [36]:
#Tail
df.tail(1)

[Row(a=3, b=4.0, c='string3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 3, 1, 12, 0))]

In [37]:
# To Pandas
df.toPandas()

/home/limoo/anaconda3/envs/spark/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-02-01 12:00:00
2,3,4.0,string3,2000-03-01,2000-03-01 12:00:00


In [43]:
#Column Operations
df.withColumn('UPPER_C',upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|UPPER_C|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01|2000-02-01 12:00:00|STRING2|
|  3|4.0|string3|2000-03-01|2000-03-01 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



In [44]:
#Filter
df.filter(df.b =='4.0').show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  3|4.0|string3|2000-03-01|2000-03-01 12:00:00|
+---+---+-------+----------+-------------------+



In [60]:
#Pandas UDF
@pandas_udf('long')
def pan_series(series: pd.Series) -> pd.Series:
    return series
df.select(pan_series(df.b)).show()

+-------------+
|pan_series(b)|
+-------------+
|            2|
|            3|
|            4|
+-------------+



In [64]:
#Map in Pandas
def pan_filter(iterator):
    for pan_df in iterator:
        yield pan_df[pan_df.a==2]
df.mapInPandas(pan_filter, schema=df.schema).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  2|3.0|string2|2000-02-01|2000-02-01 12:00:00|
+---+---+-------+----------+-------------------+



In [65]:
new_df = spark.createDataFrame([
    ['red','harrier',1,1000],
    ['black','nissan',1,900],
    ['red','probox',5,250],
    ['grey', 'axio',2,700],
    ['blue','fit',3,350]
],
schema = ['color', 'type', 'quantity','price'])

In [71]:
# Group
n=new_df.groupby('color').avg('price').show()

+-----+----------+
|color|avg(price)|
+-----+----------+
|  red|     625.0|
|black|     900.0|
| grey|     700.0|
| blue|     350.0|
+-----+----------+



In [83]:
#Mean
new_df.groupby('type').mean('price').show()

+-------+----------+
|   type|avg(price)|
+-------+----------+
|harrier|    1000.0|
| nissan|     900.0|
| probox|     250.0|
|   axio|     700.0|
|    fit|     350.0|
+-------+----------+



In [74]:
#Apply Python Function to Groups
def add_mean(pan_df):
    return pan_df.assign(price=pan_df.price + pan_df.price.mean())
new_df.groupby('color').applyInPandas(add_mean, schema=new_df.schema).show()

+-----+-------+--------+-----+
|color|   type|quantity|price|
+-----+-------+--------+-----+
|black| nissan|       1| 1800|
| blue|    fit|       3|  700|
| grey|   axio|       2| 1400|
|  red|harrier|       1| 1625|
|  red| probox|       5|  875|
+-----+-------+--------+-----+



In [90]:
#Co-Group
df1=spark.createDataFrame([
    (50020,10000,20000101),
    (50021,2055,20000102),
    (50022,1000,20000103),
    (50023,20600,20000104),
    (50024,25500,20000105),
    (50025,20500,20000105)
],('inv','amount','time'))

df2=spark.createDataFrame([
    (50021,2055,'y'),
    (50020,10000,'n'),
    (50025,20500,'n'),
    (50023,20600,'y')
],('inv','amount','deliv'))

def inv_merge(l, r):
    return pd.merge_ordered(l, r)

df1.groupby('inv').cogroup(df2.groupby('inv'))\
.applyInPandas(inv_merge, schema='inv int,amount float,time int, deliv string').show()




+-----+-------+--------+-----+
|  inv| amount|    time|deliv|
+-----+-------+--------+-----+
|50020|10000.0|20000101|    n|
|50021| 2055.0|20000102|    y|
|50022| 1000.0|20000103| null|
|50023|20600.0|20000104|    y|
|50024|25500.0|20000105| null|
|50025|20500.0|20000105|    n|
+-----+-------+--------+-----+



In [95]:
#Data Sources
#CSV
new_df.write.csv('cars',header=True)
spark.read.csv('cars',header=True).show()

+-----+-------+--------+-----+
|color|   type|quantity|price|
+-----+-------+--------+-----+
|  red|harrier|       1| 1000|
|black| nissan|       1|  900|
|  red| probox|       5|  250|
| grey|   axio|       2|  700|
| blue|    fit|       3|  350|
+-----+-------+--------+-----+



In [98]:
#Parquet
new_df.write.parquet('cars.parquet')
spark.read.parquet('cars.parquet').show()

+-----+-------+--------+-----+
|color|   type|quantity|price|
+-----+-------+--------+-----+
|black| nissan|       1|  900|
|  red|harrier|       1| 1000|
|  red| probox|       5|  250|
| grey|   axio|       2|  700|
| blue|    fit|       3|  350|
+-----+-------+--------+-----+



In [99]:
#ORC
new_df.write.orc('cars.orc')
spark.read.orc('cars.orc').show()

+-----+-------+--------+-----+
|color|   type|quantity|price|
+-----+-------+--------+-----+
|black| nissan|       1|  900|
|  red|harrier|       1| 1000|
|  red| probox|       5|  250|
| blue|    fit|       3|  350|
| grey|   axio|       2|  700|
+-----+-------+--------+-----+



In [100]:
#Dataframes to SQL Tables
new_df.createOrReplaceTempView('Cars')
spark.sql("Select count(*) from Cars").show()

+--------+
|count(1)|
+--------+
|       5|
+--------+



In [112]:
# SQL Filter
spark.sql("select price Price, type Model from Cars where color=='red'").show()

+-----+-------+
|Price|  Model|
+-----+-------+
| 1000|harrier|
|  250| probox|
+-----+-------+



In [116]:
#UDFs with SQL
@pandas_udf('integer')
def new_price(price:pd.Series) -> pd.Series:
    return price + 1
spark.udf.register("new_price", new_price)
spark.sql("select new_price(price) New_Price from Cars").show()

23/04/28 18:24:10 WARN SimpleFunctionRegistry: The function new_price replaced a previously registered function.


+---------+
|New_Price|
+---------+
|     1001|
|      901|
|      251|
|      701|
|      351|
+---------+



In [120]:
#Expressions
new_df.selectExpr('new_price(price)').show()
new_df.select(expr('count(*) < 0')).show()

+----------------+
|new_price(price)|
+----------------+
|            1001|
|             901|
|             251|
|             701|
|             351|
+----------------+

+--------------+
|(count(1) < 0)|
+--------------+
|         false|
+--------------+

